In [3]:
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
import fitz
from langchain_google_genai import ChatGoogleGenerativeAI



In [4]:
# llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0.6)
llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7)


In [5]:
with open("D:\\MCQ-GENERATOR\\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [6]:
print(RESPONSE_JSON)

{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [8]:
mcq_prompt=PromptTemplate(
    template=TEMPLATE,
    input_variables=['text','number','subject','tone','RESPONSE_JSON']
)


In [17]:
mcq_chain=LLMChain(llm=llm,prompt=mcq_prompt,output_key="quiz",verbose=True)

In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
eval_prompt_temp=PromptTemplate(
    template=TEMPLATE2,
    input_variables=['subject','quiz']
)

In [20]:
review_chain=LLMChain(llm=llm,prompt=eval_prompt_temp,output_key="review",verbose=True)

In [21]:
generate_evaluate_chain=SequentialChain(chains=[mcq_chain,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [22]:
messi_path=r"D:\MCQ-GENERATOR\data.txt"

In [23]:
with open(messi_path,'r') as file:
    text=file.read()


In [25]:
print(text)

From Wikipedia, the free encyclopedia
"Messi" redirects here. For other uses, see Messi (disambiguation).
Lionel Messi

Messi with Argentina at the 2022 FIFA World Cup
Personal information
Full name	Lionel Andrés Messi[1]
Date of birth	24 June 1987 (age 36)[1]
Place of birth	Rosario, Argentina
Height	1.70 m (5 ft 7 in)[1]
Position(s)	Forward, attacking midfielder
Team information
Current team	Inter Miami
Number	10
Youth career
1992–1995	Grandoli
1995–2000	Newell's Old Boys
2000–2003	Barcelona
Senior career*
Years	Team	Apps	(Gls)
2003–2004	Barcelona C	10	(5)
2004–2005	Barcelona B	22	(6)
2004–2021	Barcelona	520	(474)
2021–2023	Paris Saint-Germain	58	(22)
2023–	Inter Miami	16	(11)
International career‡
2004–2005	Argentina U20	18	(14)
2008	Argentina U23	5[α]	(2)
2005–	Argentina	180	(106)
Medal record
Signature
Lionel Messi signature
*Club domestic league appearances and goals, correct as of 4:00, 19 May 2024 (UTC)
‡ National team caps and goals, correct as of 02:35, 22 November 2023 (UTC)


In [26]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{RESPONSE_JSON}\n\n'), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AFDF756750>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002AFDFA58990>, default_metadata=()), output_key='quiz'), LLMChain(verbos

In [27]:
type(RESPONSE_JSON)

dict

In [28]:
json_format=json.dumps(RESPONSE_JSON)

In [29]:
type(json_format)

str

In [30]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text": text,
        "number": 10,
        "subject": "Lionel Messi",
        "tone": "Simple",
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )

d:\MCQ-GENERATOR\mcq_generator\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:From Wikipedia, the free encyclopedia
"Messi" redirects here. For other uses, see Messi (disambiguation).
Lionel Messi

Messi with Argentina at the 2022 FIFA World Cup
Personal information
Full name	Lionel Andrés Messi[1]
Date of birth	24 June 1987 (age 36)[1]
Place of birth	Rosario, Argentina
Height	1.70 m (5 ft 7 in)[1]
Position(s)	Forward, attacking midfielder
Team information
Current team	Inter Miami
Number	10
Youth career
1992–1995	Grandoli
1995–2000	Newell's Old Boys
2000–2003	Barcelona
Senior career*
Years	Team	Apps	(Gls)
2003–2004	Barcelona C	10	(5)
2004–2005	Barcelona B	22	(6)
2004–2021	Barcelona	520	(474)
2021–2023	Paris Saint-Germain	58	(22)
2023–	Inter Miami	16	(11)
International career‡
2004–2005	Argentina U20	18	(14)
2008	Argentina U23	5[α]	(2)
2005–	Argentina	180	(106)
Medal record
Signature
Lionel Messi signature
*Club domestic league appearances and goals, correc

In [31]:
quiz=response.get('quiz')

In [32]:
type(quiz)

str

In [33]:
quiz

'{"1": {"mcq": "In what year was Lionel Messi born?", "options": {"a": "1985", "b": "1987", "c": "1989", "d": "1991"}, "correct": "b"}, "2": {"mcq": "Which club did Messi make his professional debut for?", "options": {"a": "Barcelona", "b": "Real Madrid", "c": "Manchester United", "d": "Chelsea"}, "correct": "a"}, "3": {"mcq": "How many Ballon d\'Or awards has Messi won?", "options": {"a": "5", "b": "6", "c": "7", "d": "8"}, "correct": "d"}, "4": {"mcq": "What is Messi\'s nickname?", "options": {"a": "The Atomic Flea", "b": "The Little Magician", "c": "The Argentine Maestro", "d": "The Golden Boy"}, "correct": "a"}, "5": {"mcq": "Which country does Messi represent internationally?", "options": {"a": "Spain", "b": "Argentina", "c": "Brazil", "d": "Italy"}, "correct": "b"}, "6": {"mcq": "What is Messi\'s current club?", "options": {"a": "Barcelona", "b": "Real Madrid", "c": "Paris Saint-Germain", "d": "Manchester City"}, "correct": "c"}, "7": {"mcq": "How many goals has Messi scored for 

In [34]:
# Remove the leading '### RESPONSE_JSON\n'
json_str = quiz.lstrip('### RESPONSE_JSON\n')

In [35]:
final_quiz=json.loads(json_str)

In [36]:
type(final_quiz)

dict

In [ ]:
final_quiz

{'1': {'mcq': 'When was Lionel Messi born?',
  'options': {'a': '1985', 'b': '1987', 'c': '1989', 'd': '1991'},
  'correct': 'b'},
 '2': {'mcq': 'Which club did Messi make his professional debut for?',
  'options': {'a': 'Real Madrid',
   'b': 'FC Barcelona',
   'c': 'Manchester United',
   'd': 'Chelsea'},
  'correct': 'b'},
 '3': {'mcq': "How many Ballon d'Or awards has Messi won?",
  'options': {'a': '5', 'b': '6', 'c': '7', 'd': '8'},
  'correct': 'd'},
 '4': {'mcq': "What is Messi's nickname?",
  'options': {'a': 'The Flea',
   'b': 'The Magician',
   'c': 'The GOAT',
   'd': 'The King'},
  'correct': 'a'},
 '5': {'mcq': 'Which national team does Messi play for?',
  'options': {'a': 'Brazil', 'b': 'Argentina', 'c': 'Spain', 'd': 'Portugal'},
  'correct': 'b'},
 '6': {'mcq': 'How many goals did Messi score in the 2014 FIFA World Cup?',
  'options': {'a': '4', 'b': '6', 'c': '8', 'd': '10'},
  'correct': 'b'},
 '7': {'mcq': "What is Messi's current club?",
  'options': {'a': 'FC Bar

: 

In [37]:
final_quiz.items()

dict_items([('1', {'mcq': 'In what year was Lionel Messi born?', 'options': {'a': '1985', 'b': '1987', 'c': '1989', 'd': '1991'}, 'correct': 'b'}), ('2', {'mcq': 'Which club did Messi make his professional debut for?', 'options': {'a': 'Barcelona', 'b': 'Real Madrid', 'c': 'Manchester United', 'd': 'Chelsea'}, 'correct': 'a'}), ('3', {'mcq': "How many Ballon d'Or awards has Messi won?", 'options': {'a': '5', 'b': '6', 'c': '7', 'd': '8'}, 'correct': 'd'}), ('4', {'mcq': "What is Messi's nickname?", 'options': {'a': 'The Atomic Flea', 'b': 'The Little Magician', 'c': 'The Argentine Maestro', 'd': 'The Golden Boy'}, 'correct': 'a'}), ('5', {'mcq': 'Which country does Messi represent internationally?', 'options': {'a': 'Spain', 'b': 'Argentina', 'c': 'Brazil', 'd': 'Italy'}, 'correct': 'b'}), ('6', {'mcq': "What is Messi's current club?", 'options': {'a': 'Barcelona', 'b': 'Real Madrid', 'c': 'Paris Saint-Germain', 'd': 'Manchester City'}, 'correct': 'c'}), ('7', {'mcq': 'How many goals h

In [38]:
for i in final_quiz:
    print(i)

1
2
3
4
5
6
7
8
9
10


In [39]:
# options=[]
# for i in final_quiz:
#     for a,b in final_quiz.get(i)['options'].items():
#         options.append(f"{a}). {b}")
#         # options.append(a,b)


In [40]:
quiz_table_data=[]
for key,value in final_quiz.items():
    mcq=value['mcq']

    options= " | ".join(
        [
            f"{a}: {b}"
            for a,b in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"Question no.":key,"MCQ":mcq,"Choices":options,"Correct":correct})

In [41]:
quiz_table_data

[{'Question no.': '1',
  'MCQ': 'In what year was Lionel Messi born?',
  'Choices': 'a: 1985 | b: 1987 | c: 1989 | d: 1991',
  'Correct': 'b'},
 {'Question no.': '2',
  'MCQ': 'Which club did Messi make his professional debut for?',
  'Choices': 'a: Barcelona | b: Real Madrid | c: Manchester United | d: Chelsea',
  'Correct': 'a'},
 {'Question no.': '3',
  'MCQ': "How many Ballon d'Or awards has Messi won?",
  'Choices': 'a: 5 | b: 6 | c: 7 | d: 8',
  'Correct': 'd'},
 {'Question no.': '4',
  'MCQ': "What is Messi's nickname?",
  'Choices': 'a: The Atomic Flea | b: The Little Magician | c: The Argentine Maestro | d: The Golden Boy',
  'Correct': 'a'},
 {'Question no.': '5',
  'MCQ': 'Which country does Messi represent internationally?',
  'Choices': 'a: Spain | b: Argentina | c: Brazil | d: Italy',
  'Correct': 'b'},
 {'Question no.': '6',
  'MCQ': "What is Messi's current club?",
  'Choices': 'a: Barcelona | b: Real Madrid | c: Paris Saint-Germain | d: Manchester City',
  'Correct': '

In [43]:
df=pd.DataFrame(quiz_table_data)

In [44]:
df.to_csv("Question.csv",index=False)

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Compilation Tokens:{cb.completion_tokens}")

print(f"Prompt Tokens:{cb.prompt_tokens}")

print(f"Total cost:{cb.total_cost}")


In [1]:
LOG_FILE="{datetime.now().strftime('%M_%d_%Y_%H_%S')}.log"


In [2]:
LOG_FILE

"{datetime.now().strftime('%M_%d_%Y_%H_%S')}.log"

In [10]:
from datetime import  datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_24_2024_08_55_13'